In [69]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum
from scipy.integrate import quad

yu.setpath('lhapdf')

if "_has_run_once" not in globals():
    import os, sys, ctypes
    _has_run_once = True
    path='/p/project1/ngff/li47/code/LHAPDF/build/lib/'
    # path='/p/project1/ngff/li47/code/temp/LHAPDF/build/lib/'
    sys.path.insert(0, f'{path}python3.13/site-packages')
    ctypes.CDLL(f'{path}libLHAPDF.so', mode=ctypes.RTLD_GLOBAL)
    
    'python3.13/site-packages'
    import lhapdf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
def j2dic(j):
    if j == "jg":
        return {21: 1}
    if j == "jtot":
        return {**j2dic('jq;+'),**j2dic('jg')}
    
    pdg = {"u":2, "d":1, "s":3, "c":4, "g":21}
    bases = {
        "ju": {"u":1},
        "jd": {"d":1},
        "js": {"s":1},
        "jc": {"c":1},
        "j+": {"u":1, "d":1},
        "j-": {"u":1, "d":-1},
        "jq": {"u":1, "d":1, "s":1, "c":1},
        "jv1": {"u":1, "d":-1},
        "jv2": {"u":1, "d":1, "s":-2},
        "jv3": {"u":1, "d":1, "s":1, "c":-3},
    }

    base, t = j.split(";")
    result = {}
    for f, w in bases[base].items():
        pid = pdg[f]
        if t == "q":
            result[pid] = result.get(pid, 0) + w
        elif t == "b":
            result[-pid] = result.get(-pid, 0) + w
        elif t == "+":
            result[pid] = result.get(pid, 0) + w
            result[-pid] = result.get(-pid, 0) + w
        elif t == "-":
            result[pid] = result.get(pid, 0) + w
            result[-pid] = result.get(-pid, 0) - w
    return result

j2dic('jtot')

def xfx(pdf,j,x):
    Q=2.0
    return np.sum([c*pdf.xfxQ(ind,x,Q) for ind,c in j2dic(j).items()])
def getMoment(pdf,j,n=1):
    func = lambda x: x**(n-1) * xfx(pdf,j,x)
    val, _ = quad(func, 1e-6, 1.0, epsabs=1e-6, epsrel=1e-4)
    return val
def getMoment_me(replicas,j,n=1):
    dat=np.array([getMoment(pdf,j,n=n) for pdf in replicas])
    m,e=dat.mean(),dat.std(ddof=1)
    return m,e

In [101]:
js=['jtot','jq;+','jg','ju;+','jd;+','js;+','jc;+','jv1;+','jv2;+','jv3;+','j+;+','j-;+']
label2j2me=yu.load_pkl_reg('label2j2me_avgx')

if label2j2me is None:
    labels=['JAM22-PDF_proton_nlo','NNPDF40_nnlo_as_01180']
    label2replicas={label:lhapdf.getPDFSet(label).mkPDFs() for label in labels}
    
    label2j2me=defaultdict(dict)
    tasks=list(product(labels,js))
    for i,(label,j) in enumerate(tasks):
        print(f'{i}/{len(tasks)}',label,j,end='                \r')
        label2j2me[label][j]=getMoment_me(label2replicas[label],j)    
    yu.save_pkl_reg('label2j2me_avgx',label2j2me)

df=[[yu.un2str(label2j2me[label][j][0],label2j2me[label][j][1]) for label in labels] for j in js]
df=pd.DataFrame(df,index=js,columns=labels)
df

,JAM22-PDF_proton_nlo,NNPDF40_nnlo_as_01180
jtot,0.99913(96),0.9999969(47)
jq;+,0.5984(66),0.6005(26)
jg,0.4008(66),0.3995(26)
ju;+,0.3590(33),0.3468(26)
jd;+,0.2047(44),0.1934(32)
js;+,0.0262(98),0.0483(34)
jc;+,0.00845(13),0.0119(22)
jv1;+,0.1542(35),0.1533(41)
jv2;+,0.511(25),0.4435(99)
jv3;+,0.5646(71),0.5527(92)


In [98]:
js=['jtot','jq;+','jg','ju;+','jd;+','js;+','jc;+','jv1;+','jv2;+','jv3;+','j+;+','j-;+']
label2j2me=yu.load_pkl_reg('label2j2me_avgx_helicity')

if label2j2me is None:
    labels=['NNPDFpol20_nnlo_as_01180_mhou']
    label2replicas={label:lhapdf.getPDFSet(label).mkPDFs() for label in labels}
    
    label2j2me=defaultdict(dict)
    tasks=list(product(labels,js))
    for i,(label,j) in enumerate(tasks):
        print(f'{i}/{len(tasks)}',label,j,end='                \r')
        label2j2me[label][j]=getMoment_me(label2replicas[label],j)    
    yu.save_pkl_reg('label2j2me_avgx_helicity',label2j2me)

df=[[yu.un2str(label2j2me[label][j][0],label2j2me[label][j][1]) for label in labels] for j in js]
df=pd.DataFrame(df,index=js,columns=labels)
df

,NNPDFpol20_nnlo_as_01180_mhou
jtot,0.127(12)
jq;+,0.0920(78)
jg,0.035(12)
ju;+,0.1488(93)
jd;+,-0.0451(54)
js;+,-0.0077(88)
jc;+,-0.0040(15)
jv1;+,0.194(13)
jv2;+,0.119(24)
jv3;+,0.1081(76)


# backup

In [5]:
# x^n
jqs=['ju','jd','js','jc']
def j2cinds(j):
    if j=='jtot':
        return j2cinds('jq')+j2cinds('jg')
    if j=='jv1':
        t=j2cinds('jd')
        t=[(-c,ind) for c,ind in t]
        return j2cinds('ju')+t
    if j=='jv2':
        t=j2cinds('js')
        t=[(-2*c,ind) for c,ind in t]
        return j2cinds('ju')+j2cinds('jd')+t
    if j=='jv3':
        t=j2cinds('jc')
        t=[(-3*c,ind) for c,ind in t]
        return j2cinds('ju')+j2cinds('jd')+j2cinds('js')+t
    if j in ['jg']:
        return [(1,21)]
    if j in jqs:
        ind=[2,1,3,4][jqs.index(j)]
        return [(1,ind),(1,-ind)]
    if j[:2]=='jq':
        return [cind for jq in jqs for cind in j2cinds(jq+j[2:])]

def pdf_moment(pdf, cinds, n=1):
    Q=2.0
    func = lambda x: x**(n-1) * sum(c*pdf.xfxQ(ind, x, Q) for c,ind in cinds)
    val, _ = quad(func, 1e-6, 1.0, epsabs=1e-6, epsrel=1e-4)
    return val
    
js=['jtot','jq','jg','ju','jd','js','jc','jv1','jv2','jv3']
j2me={}
for n in [1,2,3]:
    for j in js:
        dat=[pdf_moment(pdf, j2cinds(j),n=n) for pdf in replicas]
        dat=np.array(dat)
        j2me[(f'<x^{n}>',j)]=dat.mean(),dat.std(ddof=1)

yu.save_pkl_reg(f'j2me_x123_{PDF_SET}',j2me)

for key in j2me.keys():
    m,e=j2me[key]
    print(key,yu.un2str(m,e))

('<x^1>', 'jtot') 0.99913(96)
('<x^1>', 'jq') 0.5984(66)
('<x^1>', 'jg') 0.4008(66)
('<x^1>', 'ju') 0.3590(33)
('<x^1>', 'jd') 0.2047(44)
('<x^1>', 'js') 0.0262(98)
('<x^1>', 'jc') 0.00845(13)
('<x^1>', 'jv1') 0.1542(35)
('<x^1>', 'jv2') 0.511(25)
('<x^1>', 'jv3') 0.5646(71)
('<x^2>', 'jtot') 0.1815(16)
('<x^2>', 'jq') 0.1386(18)
('<x^2>', 'jg') 0.0429(22)
('<x^2>', 'ju') 0.09346(85)
('<x^2>', 'jd') 0.0403(13)
('<x^2>', 'js') 0.0043(28)
('<x^2>', 'jc') 0.000514(25)
('<x^2>', 'jv1') 0.0531(11)
('<x^2>', 'jv2') 0.1252(71)
('<x^2>', 'jv3') 0.1365(19)
('<x^3>', 'jtot') 0.0579(10)
('<x^3>', 'jq') 0.0494(10)
('<x^3>', 'jg') 0.00853(83)
('<x^3>', 'ju') 0.03524(37)
('<x^3>', 'jd') 0.01243(65)
('<x^3>', 'js') 0.0017(15)
('<x^3>', 'jc') 0.0000714(54)
('<x^3>', 'jv1') 0.02281(60)
('<x^3>', 'jv2') 0.0443(38)
('<x^3>', 'jv3') 0.0491(10)
